# Natural Language Processing
## 3️⃣ Emotion Recognition

### Navie Bayes Classification

$$P(Emotion|Text) = \frac{P(Text|Emotion) * P(Emotion)}{P(Text)}$$

We can use **Bayes Theorem** to predict the probability of occurrence of each emotions in text.

#### Likelihood of text

The **likelihood of Text** can be calculated by frequency of words.   
$$P(Word | Emotion) = \frac{\text{(Frequency of Word in the emotion)}}{\text{(Frequency of every word in the emotion)}}$$

For example: 
$$P(fun | Happy) = \frac{\text{(Frequency of "fun" in sentences that expesses happiness)}}{\text{(Frequency of every word in sentences that expresses happiness)}}$$

#### Likelihood of Emotion

The **likelihood of Emotion** is calculated as the ratio of sentences expressing the emotion within the entire data.
$$P(Emotion)=\frac{\text{(Number of sentences that express the emotion)}}{\text{(Number of sentences in entire data)}}$$

After calculating $P(Emotion|Text)$ for every emotions, the model predicts the emotion with the greatest value among each probability as the emotion in the corresponding sentence.

#### Likelihood of Sentence

The **likelihood of Sentence** is calculated by multiplying likelihood of each words in the sentence and likelihood of the emotion. For detailed description, click [here](https://web.stanford.edu/~jurafsky/slp3/4.pdf).
$$\text{Probability that sentence expresses Emotion} = P(Word_1 | Emotion) * ... * P(Word_n | Emotion) * P(Emotion)$$

#### Smoothing

In the case of words that **do not exist in the learning data**, the likelihood  becomes zero. To prevent this, the frequency of words that do not exist in the learning data must be corrected by **smoothing** as below :

$$P(Word | Emotion) = \frac{\text{(Frequency of Word in the emotion)} + 1}{\text{(Frequency of every word in the emotion)} + 1}$$

#### Calculating the probability using the log

To calculate likelihood of a **sentence**, model multiplies likelihoods of several words, which makes the result **too small** for the computer to express the value any longer. So, the model uses **log** to express the likelihood as below :

$$log(P(Sentence | Emotion)) = log(P(Word_1 | Emotion)) + ... + log(P(Word_n | Emotion)) + log(P(Emotion))$$

In [1]:
import pandas as pd
import numpy as np

# calculates the frequency at which each word appears in a sentence representing corresponding emotion,
# and put that frequency into partial_freq.
def cal_partial_freq(texts, emotion):
    filtered_texts = texts[texts['emotion'] == emotion]
    filtered_texts = filtered_texts['sentence']
    partial_freq = dict()

    for line in filtered_texts:
        for word in line.rstrip().split():
            if word not in partial_freq:
                partial_freq[word] = 1
            else :
                partial_freq[word] += 1
    
    return partial_freq

# calculates the sum of frequencies of every words in partial_freq
def cal_total_freq(partial_freq):
    total = 0
    for word, freq in partial_freq.items() :
        total += freq
    return total

# return log likelihood of coressponding emotion
def cal_prior_prob(data, emotion):
    filtered_texts = data[data['emotion'] == emotion]
    return np.log(len(filtered_texts) / len(data))

# Predict which emotion does corresponding sentence 'sent' expresses.
def predict_emotion(sent, data):
    emotions = ['anger', 'love', 'sadness', 'fear', 'joy', 'surprise']
    predictions = []
    train_txt = pd.read_csv(data, delimiter=';', header=None, names=['sentence', 'emotion'])

    # Save each emotion's lieklihood of sent to predictions.
    for emotion in emotions:
        emotion_prob = cal_prior_prob(train_txt, emotion)
        emotion_counter = cal_partial_freq(train_txt, emotion)
        prob = 0
        for word in sent.split() :
            # Set smoothing to 10.
            prob += np.log((emotion_counter[word] + 10) / (cal_total_freq(emotion_counter) + 10))
        prob += emotion_prob
        predictions.append((emotion, prob))
        predictions.sort(key = lambda a: a[1])
    return predictions[-1]

test_sent = "i really want to go and enjoy this party"
predicted = predict_emotion(test_sent, "emotions_train.txt")
print(predicted)


('surprise', -49.413280143234715)


### Implementing Naive Bayes classification using scikit-learn

To implement Naive Bayes classification using scikit-learn, sentences in data for training must be transformed into **vectors** using **CountVectorizer** from scikit-learn.

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

doc = ["i am very happy", "this product is really great"]
emotion = ["happy", "excited"]

cv = CountVectorizer()
csr_doc_matrix = cv.fit_transform(doc)

print(csr_doc_matrix)

  (0, 0)	1
  (0, 7)	1
  (0, 2)	1
  (1, 6)	1
  (1, 4)	1
  (1, 3)	1
  (1, 5)	1
  (1, 1)	1


After transforming the data, we can train the model as below :

In [3]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

clf.fit(csr_doc_matrix, emotion)

MultinomialNB()

To predict test sentence, we must transform the sentence for test into vector too.

In [5]:
test_doc = ["i am really great"]

transformed_test = cv.transform(test_doc)
pred = clf.predict(transformed_test)

print(pred)

['excited']
